## 시간 흐름에 따른 개발 언어 인기도 분석 

- 오늘날 여전히 사용되는 가장 오랜된 개발 언어는 1957년에 개발된 포트란(FORTRAN)이다. 이후로 많은 개발 언어가 발전해왔다. 그런데 가장 인기 있는 개발 언어는 무엇일까? 
- 이 뜨거운 질문에 대한 대한 답은 스택 오버플로우(Stack Overflow)에서 도움을 받을 수 있다. 스택 오버플로우의 각 게시물은 태그가 함께 제공되며, 개발 언어 이름이 태그가 될 수 있다. 
- 어떤 언어가 가장 인기 있는지 알아내려면 스택 오버플로우에서 각 언어로 태그가 지정된 게시물 수를 계산하면 된다.(가장 많은 게시물을 가진 언어가 바로 인기가 많은 개발 언어라고 할 수 있다.)

### 여기서 배울 내용 
1. 맷플롯립(Matplotlib)으로 데이터를 시각화 하고 차트를 만드는 법
2. 판다스로 데이터를 피벗, 그룹화 및 조작하여 원하는 형식으로 가져오는 방법 
3. 타임스템프 및 시계열 데이터로 작업하는 방법
4. 원하는 대로 꺾은선형 차트의 스타일을 사용자 지정하는 방법

### 데이터를 얻는 법
- 제공된 .csv 파일을 사용하거나 (선택적으로) StackExchange에서 SQL 쿼리를 실행하여 새로운 (가장 신선한?) 데이터를 얻어보아라.
- 제공되는 해당 링크 따라 [StackExchange](https://data.stackexchange.com/stackoverflow/query/675441/popular-programming-languages-per-over-time-eversql-com)에서 쿼리를 실행하여 .csv 파일을 얻어보아라/
- SQL 쿼리문
<code>
select dateadd(month, datediff(month, 0, q.CreationDate), 0) m, TagName, count(*)
from PostTags pt
join Posts q on q.Id=pt.PostId
join Tags t on t.Id=pt.TagId
where TagName in ('java','c','c++','python','c#','javascript','assembly','php','perl','ruby','visual basic','swift','r','object-c','scratch','go','swift','delphi')
and q.CreationDate < dateadd(month, datediff(month, 0, getdate()), 0)
group by dateadd(month, datediff(month, 0, q.CreationDate), 0), TagName
order by dateadd(month, datediff(month, 0, q.CreationDate), 0)
</code>

- 이미 제공된 CSV 파일을 사용해도 무관함

### 1. 데이터 열기 및 탐색

In [1]:
# 필요한 패키지 부르기 
import pandas as pd

In [3]:
# 데이터 판다스로 읽고, 컬럼명 지정하기 
df = pd.read_csv('QueryResults.csv')
df.columns = ['DATE', 'TAG', 'POSTS']
df

,DATE,TAG,POSTS
0,2008-07-01 00:00:00,c#,3
1,2008-08-01 00:00:00,assembly,8
2,2008-08-01 00:00:00,javascript,162
3,2008-08-01 00:00:00,c,85
4,2008-08-01 00:00:00,python,124
...,...,...,...
1986,2020-07-01 00:00:00,r,5694
1987,2020-07-01 00:00:00,go,743
1988,2020-07-01 00:00:00,ruby,775
1989,2020-07-01 00:00:00,perl,182


In [2]:
# 데이터 불러오기 또다른 방법 - 여기서 헤더 행을 0으로 설정하면 열 자체의 이름으로 대처할 수 있음
df = pd.read_csv('QueryResults.csv', names=['DATE', 'TAG', 'POSTS'], header=0)
df

,DATE,TAG,POSTS
0,2008-07-01 00:00:00,c#,3
1,2008-08-01 00:00:00,assembly,8
2,2008-08-01 00:00:00,javascript,162
3,2008-08-01 00:00:00,c,85
4,2008-08-01 00:00:00,python,124
...,...,...,...
1986,2020-07-01 00:00:00,r,5694
1987,2020-07-01 00:00:00,go,743
1988,2020-07-01 00:00:00,ruby,775
1989,2020-07-01 00:00:00,perl,182


In [3]:
# 현제 데이터의 행과 열의 개수 파악
df.shape

(1991, 3)

- 해당 csv 파일의 데이터의 총개수는 아래와 같다.
    - 행: 1,991 건
    - 열: 3건

In [4]:
# 각 열에 있는 항목의 수 세기 
df.count()

DATE     1991
TAG      1991
POSTS    1991
dtype: int64

- 만약 열에 있는 항목의 수를 세기 위해서 `.count()`를 사용할 수 있다. count()는 실제로 각 열에 있는 NaN이 아닌 값의 개수를 알려준다.